In [1]:
import json
import os
from os.path import join
from pathlib import Path

import matplotlib.pyplot as plt
import nibabel as nib
import numpy as np
import pandas as pd
from dn_utils.behavioral_models import load_behavioral_data
from nilearn.input_data import NiftiSpheresMasker

In [2]:
path_root = os.environ.get('DECIDENET_PATH')

path_derivatives = join(path_root, 'data/main_fmri_study/derivatives') 
path_sourcedata = join(path_root, 'data/main_fmri_study/sourcedata') 

path_beh = join(path_sourcedata, 'behavioral')
path_bsc = join(path_derivatives, 'bsc')
path_parcellations = join(path_derivatives, 'parcellations')
path_data_paths = join(path_derivatives, 'data_paths')

path_betamaps = join(path_bsc, 'betamaps')
path_betaseries = join(path_bsc, 'betaseries')

Path(path_betaseries).mkdir(exist_ok=True)

In [3]:
atlas = '300_ROI_Set'
roi_table_fname = 'ROIs_300inVol_MNI_allInfo.txt'

masker_kwargs = {
    'allow_overlap': True, 
    'standardize': True, 
    'detrend': False, 
    'high_pass': None,
    'low_pass': None
}

use_mask = False

In [4]:
# Load ROI data
df_roi = pd.read_csv(join(path_parcellations, atlas, roi_table_fname), sep=' ')

# Load behavioral data
beh, meta = load_behavioral_data(path_beh, verbose=False)
n_subjects = beh.shape[0]
n_conditions = beh.shape[1]
n_trials = beh.shape[2]

# Load masks
with open(join(path_data_paths, 'mask_filenames.json'), 'r') as f:
    mask_files = json.loads(f.read())

# Load betamats
imgs = {'prlrew': [], 'prlpun': []}
for con_idx, con in enumerate(meta['dim2']):
    for sub_idx, sub in enumerate(meta['dim1']):
        img_fname = f'sub-{sub}_task-prl{con}_betamaps.nii.gz' 
        imgs[f'prl{con}'].append(nib.load(join(path_betamaps, img_fname)))

In [5]:
n_rois = len(df_roi)
seeds = [tuple(coords[1]) for coords in df_roi[['x', 'y', 'z']].iterrows()]

maskers = {}

for radius in df_roi['radius(mm)'].unique():
    # Select ROIs with given radius
    roi_indices = np.flatnonzero(df_roi['radius(mm)'] == radius)
    
    # Create masker for single radius value
    masker = NiftiSpheresMasker(
        [seeds[idx] for idx in roi_indices], 
        radius=radius,                
        mask_img=None,
        **masker_kwargs
    )
    
    maskers[radius] = {'masker': masker, 'indices': roi_indices}

In [6]:
betaseries_aggregated = np.zeros((n_subjects, n_conditions, n_trials, n_rois))

for sub_idx, sub in enumerate(meta['dim1']):
    for con_idx, con in enumerate(meta['dim2']):
        print(sub, con)
        
        mask_img = nib.load(mask_files[f'prl{con}'][sub_idx])
        beta_img = imgs[f'prl{con}'][sub_idx]

        betaseries = np.zeros((n_trials, n_rois))

        for radius in maskers:
            # Get indices to insert computed timeseries into right positions
            roi_indices = maskers[radius]['indices']

            # Get right spheres masker and add right whole-brain mask 
            masker = maskers[radius]['masker']
            if use_mask:
                masker.mask_img = mask_img

            # Extract timeseries          
            betaseries[:, roi_indices] = masker.fit_transform(beta_img)
                
        betaseries_aggregated[sub_idx, con_idx] = betaseries
        
# Store betaseries
Path(join(path_betaseries, atlas)).mkdir(exist_ok=True, parents=True)
np.save(join(path_betaseries, atlas, 'betaseries_aggregated.npy'), 
        betaseries_aggregated)

m02 rew
m02 pun
m03 rew
m03 pun
m04 rew
m04 pun
m05 rew
m05 pun
m06 rew
m06 pun
m07 rew
m07 pun
m08 rew
m08 pun
m09 rew
m09 pun
m10 rew
m10 pun
m11 rew
m11 pun
m12 rew
m12 pun
m13 rew
m13 pun
m14 rew
m14 pun
m15 rew
m15 pun
m16 rew
m16 pun
m17 rew
m17 pun
m18 rew
m18 pun
m19 rew
m19 pun
m20 rew
m20 pun
m21 rew
m21 pun
m22 rew
m22 pun
m23 rew
m23 pun
m24 rew
m24 pun
m25 rew
m25 pun
m26 rew
m26 pun
m27 rew
m27 pun
m28 rew
m28 pun
m29 rew
m29 pun
m30 rew
m30 pun
m31 rew
m31 pun
m32 rew
m32 pun
m33 rew
m33 pun
